In [1]:
import gym
import collections
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, Model
experience_buffer = collections.deque([])

env = gym.make("ALE/Breakout-v5")

N_ACTIONS = env.action_space.n
RANGE = int(env.observation_space.high_repr) - int(env.observation_space.low_repr)
SHAPE = env.observation_space.shape
K_FRAMES = 4

model = tf.keras.applications.efficientnet.EfficientNetB0(
    include_top=True,
    weights=None,
    input_shape=SHAPE,
    classes=N_ACTIONS,
    classifier_activation='softmax'
)

total_params = sum([np.prod(w.get_shape().as_list()) for w in model.trainable_weights])
print(total_params)


/home/marcello/github/AtariGym/env/lib/python3.8/site-packages/flatbuffers/compat.py:19: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
A.L.E: Arcade Learning Environment (version 0.7.4+069f8bd)
[Powered by Stella]
/home/marcello/github/AtariGym/env/lib/python3.8/site-packages/gym/utils/seeding.py:138: DeprecationWarning: WARN: Function `hash_seed(seed, max_bytes)` is marked as deprecated and will be removed in the future. 
  deprecation(
/home/marcello/github/AtariGym/env/lib/python3.8/site-packages/gym/utils/seeding.py:175: DeprecationWarning: WARN: Function `_bigint_from_bytes(bytes)` is marked as deprecated and will be removed in the future. 
  deprecation(
2022-04-06 00:36:36.510420: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-06 00:36:36

4012672


In [2]:
def preprocess_observation(obs):
    return (obs / RANGE)

In [12]:
obs = np.zeros([K_FRAMES, *SHAPE])

for _ in range(int(1e6)):                           # n° of episodes

      observation, info = env.reset(seed=42, return_info=True)
      action = env.action_space.sample()

      for _ in range(int(1e3)):                     # n° of frames per episode
            
            # legal_action_list = env.action_space
            i = 0
            k_reward = 0

            while i < K_FRAMES and not done:
                  observation, reward, done, info = env.step(action)
                  obs[i] = observation
                  i += 1
                  k_reward += reward

            p_obs = preprocess_observation(obs)

            action = max(model(p_obs)) # CUDDN dio merda
            print(action)

            observation, reward, done, info = env.step(0)

            if done:
                observation, info = env.reset(return_info=True)
            # break
            break
      break

env.close()

/home/marcello/github/AtariGym/env/lib/python3.8/site-packages/gym/utils/seeding.py:138: DeprecationWarning: WARN: Function `hash_seed(seed, max_bytes)` is marked as deprecated and will be removed in the future. 
  deprecation(
/home/marcello/github/AtariGym/env/lib/python3.8/site-packages/gym/utils/seeding.py:175: DeprecationWarning: WARN: Function `_bigint_from_bytes(bytes)` is marked as deprecated and will be removed in the future. 
  deprecation(
2022-04-06 00:42:34.453197: E tensorflow/stream_executor/cuda/cuda_dnn.cc:373] Could not create cudnn handle: CUDNN_STATUS_NOT_INITIALIZED
2022-04-06 00:42:34.453264: E tensorflow/stream_executor/cuda/cuda_dnn.cc:382] Possibly insufficient driver version: 510.47.3
2022-04-06 00:42:34.453293: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at conv_ops.cc:1120 : UNIMPLEMENTED: DNN library is not found.


UnimplementedError: Exception encountered when calling layer "stem_conv" (type Conv2D).

DNN library is not found. [Op:Conv2D]

Call arguments received:
  • inputs=tf.Tensor(shape=(4, 211, 161, 3), dtype=float32)